Решение интегрального уравнения первого рода
Мой вариант -- 4
Ядро = sin(2sx)

Дано ур-ие 22 (методичка стр. 23)

1) Сводим его к СЛАУ 24. Для его решения применяем метод регуляризации
Для отладки используем функции z_1(s) = 1, z_2(s) = s(1 - s), по которым строятся соответствующие правые части u(x)

2) Решаем уравнение 27 c однородными граничными условиями z(0) = z(1) = 0 (стр. 26)
Уравнение Эйлера 32, его аппроксимация приведена на стр. 27, в итоге приводим к СЛАУ, для решения которых применяем
первый способ и отлаживаем на тех же функциях

Можем регулировать поведение функций q и r
u_delta -- u с учётом погрешности машинного нуля


Необходимо сравнить результаты обоих методов, привести графики и дать практические рекомендации по решению таких задач
(выбор n и параметр регуляризации)

In [119]:
import numpy as np
from typing import Tuple

from numpy import float64, exp, sin, cos, log
from scipy import integrate

In [120]:
def find_h(a: np.float64, b: np.float64, m: int) -> np.float64:
    return float64((b - a) / m)


def middle_rectangle_method(function, a: np.float64, b: np.float64) -> Tuple[np.float64, np.float64]:
    m = pow(10, 5)
    h = float64(find_h(a, b, m))
    start_point = a + h / 2
    points = []
    sum = 0

    for j in range(0, m):
        current_point = start_point + j * h
        points.append(current_point)
        sum += function(current_point)
    result = h * sum

    return result, points


def find_discrepancy(expected, actual):
    return abs(expected - actual)


def find_norm(expected, actual):
    assert expected.shape == actual.shape, f"Vectors shapes differ: {expected.shape} and {actual.shape}"
    return np.linalg.norm(expected - actual)
    return np.linalg.norm(expected - actual, ord=np.inf)

In [130]:
# See task.png
# With given K(x, s) and u(x) we should find z(x)
def form_and_solve_linear_equation_for_integral_equation(kernel, u_func, alfa):
    n = pow(10, 3)
    a = 0
    b = 1
    h = float64(find_h(a, b, n))
    start_point = a + h / 2
    points = []

    for j in range(n):
        points.append(start_point + j * h)

    C = np.zeros((n, n))
    U = np.zeros(n)
    for i in range(n):
        U[i] = u_func(points[i])
        for j in range(n):
            C[i, j] = h * kernel(points[i], points[j])

    solution = np.linalg.solve(C, U)
    C_conj = C.T
    C_reg = C_conj @ C  + alfa * np.eye(n)
    U_reg = C_conj @ U
    solution_reg = np.linalg.solve(C_reg, U_reg)

    return points, solution, solution_reg

In [ ]:
def form_and_solve_linear_euler_equation(kernel, u_func, p, r, alfa):
    n = pow(10, 3)
    a = 0
    b = 1
    h = float64(find_h(a, b, n))
    start_point = a + h / 2
    points = []

    for j in range(n):
        points.append(start_point + j * h)

    # z(0) = z(1) = 0
    kernel_1 = lambda s, t: integrate.quad(lambda x: kernel(x, s) * kernel(x, t), a, b)[0]

    C = np.zeros((n, n))
    U = np.zeros(n)
    for k in range(1, n - 1):
        U[i] = integrate.quad(lambda t: kernel(t, points[j]) * u_func(t),0, 1)[0]\
               - alfa * (-(p * 1))
        for j in range(n - 1):
            C[i, j] = h * kernel_1(points[k], points[j])

    solution = [0, *np.linalg.solve(C, U), 0]
    C_conj = C.T
    C_reg = C_conj @ C  + alfa * np.eye(n)
    U_reg = C_conj @ U
    solution_reg = [0, *np.linalg.solve(C_reg, U_reg), 0]

    return points, solution, solution_reg

In [141]:
[1, 2, 3, *np.linalg.solve(np.array([[1, 0], [0, 1]]), np.array([1, 1]))]

[1, 2, 3, 1.0, 1.0]

In [138]:
kernel = lambda x, s: sin(2 * x * s)

first_test_z_function = lambda x: 1
first_test_u_function = lambda x: (sin(x) ** 2) / x

second_test_z_function = lambda x: x * (1 - x)
second_test_u_function = lambda x: (sin(x) * (sin(x) - x * cos(x))) / (2 * x ** 3)

kernel_sasha = lambda x, s: 1 / (1 + s*x)
second_test_u_function_sasha = lambda x: (x * (x + 2) - 2 * (x + 1) * log(x + 1)) / (2 * x ** 3)

for alfa in [10 ** i for i in range(1, -13, -1)]:
# for alfa in [10 ** i for i in range(1, 10)]:  <--- doesn't work, alfa must be small
    points, actual, actual_reg = form_and_solve_linear_equation_for_integral_equation(kernel, first_test_u_function, alfa)
    expected = np.array([first_test_z_function(point) for point in points])

    points, actual, actual_reg = form_and_solve_linear_equation_for_integral_equation(kernel_sasha, second_test_u_function_sasha, alfa)
    expected = np.array([second_test_z_function(point) for point in points])

    points, actual, actual_reg = form_and_solve_linear_equation_for_integral_equation(kernel, second_test_u_function, alfa)
    expected = np.array([second_test_z_function(point) for point in points])

    print(f"Alfa = {alfa}. Norm = {find_norm(expected, actual_reg)}")

Alfa = 10. Norm = 5.660157370975473
Alfa = 1. Norm = 4.891309064973328
Alfa = 0.1. Norm = 3.27201201546221
Alfa = 0.01. Norm = 2.7899754148511633
Alfa = 0.001. Norm = 1.6309388385373746
Alfa = 0.0001. Norm = 0.5146427295398939
Alfa = 1e-05. Norm = 0.4160717974202384
Alfa = 1e-06. Norm = 0.3781823765565235
Alfa = 1e-07. Norm = 0.25139820784356864
Alfa = 1e-08. Norm = 0.2064013367609213
Alfa = 1e-09. Norm = 0.2044315898082477
Alfa = 1e-10. Norm = 0.19741272613177402
Alfa = 1e-11. Norm = 0.15506210836248516
Alfa = 1e-12. Norm = 0.10943091409387476
